In [34]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px

is_debugging_enabled = False

def debug_log(message):
    if is_debugging_enabled:
        print(f"[DEBUG] {message}")

# Auto-convert date columns
def auto_convert_dates(df):
    for col in df.columns:
        try:
            if df[col].dtype == 'object':
                df[col] = pd.to_datetime(df[col], errors='coerce')
        except Exception as e:
            print(f"Could not convert {col}: {e}")
    return df
    
# Plot charts dynamically
def plot_charts(spark_df):
    
    # Convert Spark DataFrame to Pandas
    df = spark_df.toPandas()

    print("\n *************************************************************************************************************")
    # Display table
    num_rows = 5
    styled_df = df.head(min(num_rows, len(df))).style.set_table_attributes('style="width:40%"').set_caption("Data in Raw Form").hide(axis="index")
    display(styled_df)

    if len(df) == 1 and len(df.columns) == 1:
        # Handle single aggregation result
        print("Aggregation result detected.")
        display(df)
        return

    # Automatically detect and convert date columns
    df = auto_convert_dates(df)
    print("\n *************************************************************************************************************")

    # Detect date and numeric columns in Pandas DataFrame
    date_cols = df.select_dtypes(include=['datetime', 'object']).columns
    numeric_cols = df.select_dtypes(include=['number']).columns

    # Convert potential date columns
    for col in date_cols:
        try:
            df[col] = pd.to_datetime(df[col], errors='coerce')
        except Exception:
            continue
    
    # Filter valid datetime columns
    date_cols = df.select_dtypes(include=['datetime']).columns
    
    if len(date_cols) == 0:
        print("No datetime column detected, plotting using index.")
        df.reset_index(inplace=True)
        date_col = df.index
    else:
        date_col = date_cols[0]  # Pick the first datetime column
    
    # Set axis limits
    x_min = df[date_col].min()
    x_max = df[date_col].max()

    # Plot for each numeric column
    for col in numeric_cols:
        fig = px.line(df, x=date_col, y=col, title=f'{col} Over Time')
        fig.update_layout(xaxis_range=[x_min, x_max])  # Fix axis to data range
        fig.show()




In [35]:
from pyspark.sql import SparkSession

print("Block : run_sql_command_on_spark")
def run_sql_command_on_spark(sql_command):
    spark = SparkSession.builder \
        .appName("DeltaTableReader").master("local[*]") \
        .getOrCreate()
    
    delta_table_path = "E:/Downloads/yellow_tripdata_2024-07.parquet"
    df = spark.read.format("parquet").load(delta_table_path)
    df.createOrReplaceTempView("taxi_trips")
    if is_debugging_enabled:
        print("checking the dataframe")
        df.show()
    result = spark.sql(sql_command)
    if is_debugging_enabled:
        print("result after running the SQL command")
        result.show()
    plot_charts(result)

Block : run_sql_command_on_spark


In [50]:
import os  
import base64
import json
from openai import AzureOpenAI  

endpoint = os.getenv("ENDPOINT_URL", "https://testaihubservice.openai.azure.com/")  
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o-mini")  
search_endpoint = os.getenv("SEARCH_ENDPOINT", "https://testaisearchservice830.search.windows.net/")  
search_key = os.getenv("SEARCH_KEY", "--")  
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "--")

print("Block : call_llm")

def call_llm(user_input):
    # Initialize Azure OpenAI client with key-based authentication    
    client = AzureOpenAI(  
        azure_endpoint=endpoint,  
        api_key=subscription_key,  
        api_version="2024-05-01-preview",  
    )
    prompt_content = "You are an AI assistant specialized in transforming natural language into SQL queries.The dataset includes embedding vectors representing predefined query patterns. Use these embeddings to adapt the generated SQL queries, aligning them with patterns found in the embedding dataset attached to the query. Return only the SQL output without explanations or comments"    #Prepare the chat prompt 
    chat_prompt = [
        {
            "role": "system",
            "content": f"{prompt_content}"
        },
        {
            "role": "user",
            "content": f"{user_input}"
        }
    ]
        
    # Include speech result if speech is enabled  
    messages = chat_prompt  
        
    # Generate the completion  
    completion = client.chat.completions.create(  
        model=deployment,  
        messages=messages,  
        max_tokens=800,  
        temperature=0.67,  
        top_p=0.95,  
        frequency_penalty=0,  
        presence_penalty=0,  
        stop=None,  
        stream=False,
        extra_body={
          "data_sources": [{
              "type": "azure_search",
              "parameters": {
                "filter": None,
                "endpoint": f"{search_endpoint}",
                 "index_name": "dynamic-coat-vvlt360cf7",
                "semantic_configuration": "azureml-default",
                "authentication": {
                  "type": "api_key",
                  "key": f"{search_key}"
                },
                "embedding_dependency": {
                  "type": "endpoint",
                  "endpoint": "https://testaihubservice.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview",
                  "authentication": {
                    "type": "api_key",
                    "key": f"{subscription_key}"
                  }
                },
                "query_type": "vector_simple_hybrid",
                "in_scope": True,
                "role_information": f"{prompt_content}",
                "strictness": 3,
                "top_n_documents": 5
              }
            }]
        }
    )
    
    res = completion.to_json()
    if is_debugging_enabled:
        print("res from LLM"+ res)
    res = json.loads(res)
    content = res["choices"][0]["message"]["content"]
    print("Key response from LLM " + content)

    if "SELECT" in content:
        # Remove ```sql and ``` from the content
        clean_sql = content.replace("```sql", "").replace("```", "").strip()
        if is_debugging_enabled:
            print("Cleaned up SQL command:\n" + clean_sql)
        run_sql_command_on_spark(clean_sql)
       


Block : call_llm


In [51]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Text input widget
text_input = widgets.Textarea(
    placeholder='Ask your question here...',
    layout=widgets.Layout(width='80%', height='50px')
)

# def call_llm(text):
#     print(f"I am text! Here's the input:\n{text}")
        
# Callback function to handle button click
def button_callback(b):
    with output:
        clear_output()  # Clear previous output
        text = text_input.value
        #print(f"I am clicked! Here's the input:\n{text}")
        call_llm(text)
        
        
# Button to trigger the action
button = widgets.Button(description='Search')

# Output widget to show results
output = widgets.Output()

# Attach the function to the button click event
button.on_click(button_callback)

# Display the widgets
display(text_input, button, output)


Textarea(value='', layout=Layout(height='50px', width='80%'), placeholder='Ask your question here...')

Button(description='Search', style=ButtonStyle())

Output()